In [1]:
import pandas as pd
import numpy as np
np.random.seed(1)

Paris_listings = pd.read_csv("paris_airbnb.csv")
Paris_listings = Paris_listings.loc[np.random.permutation(len(Paris_listings))]
stripped_commas = Paris_listings["price"].str.replace(',',"")
stripped_dollars = stripped_commas.str.replace('$',"")
Paris_listings["price"] = stripped_dollars.astype("float")

C:\Users\yayas\AppData\Local\Temp\ipykernel_16416\543081641.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stripped_dollars = stripped_commas.str.replace('$',"")


In [2]:
print(Paris_listings.info()) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 4740 to 5157
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    5000 non-null   object 
 1   host_acceptance_rate  0 non-null      float64
 2   host_listings_count   7999 non-null   float64
 3   latitude              8000 non-null   float64
 4   longitude             8000 non-null   float64
 5   city                  7997 non-null   object 
 6   zipcode               7930 non-null   object 
 7   state                 7977 non-null   object 
 8   accommodates          8000 non-null   int64  
 9   room_type             8000 non-null   object 
 10  bedrooms              7976 non-null   float64
 11  bathrooms             7942 non-null   float64
 12  beds                  7986 non-null   float64
 13  price                 8000 non-null   float64
 14  cleaning_fee          6250 non-null   object 
 15  security_deposit  

In [3]:
dop_columns = ["room_type","city","state","longitude","latitude","zipcode","host_response_rate","host_acceptance_rate","host_listings_count"]
Paris_listings = Paris_listings.drop(dop_columns,axis = 1)
print(Paris_listings.isnull().sum())

accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64


In [6]:
drop_columns = ["cleaning_fee","security_deposit"]
Paris_listings = Paris_listings.drop(drop_columns,axis = 1)
print(Paris_listings.isnull().sum())

accommodates          0
bedrooms             24
bathrooms            58
beds                 14
price                 0
minimum_nights        0
maximum_nights        0
number_of_reviews     0
dtype: int64


In [8]:
drop_columns = ["bedrooms","bathrooms","beds"]
Paris_listings = Paris_listings.dropna(axis = 0) #axis = 0 : ligne, axis = 1 : colonnes
print(Paris_listings.isnull().sum())

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64


In [9]:
Paris_listings.head(6)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
4740,4,1.0,1.0,2.0,65.0,2,1125,6
5606,3,2.0,1.5,2.0,98.0,5,1124,1
4824,2,0.0,1.0,1.0,65.0,10,1125,0
4205,2,1.0,1.0,1.0,45.0,5,1125,83
3228,3,1.0,1.0,2.0,65.0,5,365,5
2745,4,1.0,1.0,2.0,88.0,3,30,120


In [10]:
first_transformation = Paris_listings["maximum_nights"] - Paris_listings["maximum_nights"].mean()
normalized_col = first_transformation / first_transformation.std()

In [12]:
normalized_listings = (Paris_listings - Paris_listings.mean())/(Paris_listings.std())
normalized_listings["price"] = Paris_listings["price"]
print(normalized_listings.head(3))

      accommodates  bedrooms  bathrooms     beds  price  minimum_nights  \
4740      0.503868 -0.296884  -0.293537  0.20531   65.0       -0.184601   
5606     -0.131849  0.892605   0.843973  0.20531   98.0       -0.101183   
4824     -0.767566 -1.486372  -0.293537 -0.64526   65.0        0.037847   

      maximum_nights  number_of_reviews  
4740        1.062858          -0.564545  
5606        1.061018          -0.636924  
4824        1.062858          -0.651400  


In [19]:
from scipy.spatial import distance
first_fifth_distance = distance.euclidean(normalized_listings.iloc[0][["accommodates","bedrooms"]],normalized_listings.iloc[4][["accommodates","bedrooms"]])
print(first_fifth_distance)

0.6357172321498354


In [20]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(algorithm = "brute")

In [23]:
train_df = normalized_listings.iloc[0:6000]
test_df = normalized_listings.iloc[6000:]

#Objet de type matrice des 2 colonnes
train_features = train_df[["accommodates","bedrooms"]]

#Objet list cible
train_target = train_df["price"]

knn.fit(train_features,train_target)

KNeighborsRegressor(algorithm='brute')

In [26]:
predictions = knn.predict(test_df[["accommodates","bedrooms"]])
print(predictions)

[ 60.   81.8  81.8 ...  60.  121.2  81.8]


In [33]:
from sklearn.metrics import mean_squared_error

two_features_mse = mean_squared_error(test_df["price"],predictions,)
two_features_rmse = np.sqrt(two_features_mse)
print(two_features_mse,two_features_rmse)


6238.268920187794 78.9827127932929


In [36]:


features = ["accommodates","bedrooms","number_of_reviews","bathrooms","beds","minimum_nights","maximum_nights","number_of_reviews"]
knn.fit(train_df[features],train_df["price"])
four_predictions = knn.predict(test_df[features])
four_mse =  mean_squared_error(test_df["price"],four_predictions)
four_rmse = np.sqrt(four_mse)

print(four_mse)
print(four_rmse)


6471.841898800208
80.4477588177583
